<div style="background-color: #04D7FD; padding: 20px; text-align: left;">
    <h1 style="color: #000000; font-size: 36px; margin: 0;">Data Processing for RAG with Data Prep Kit (RAY)</h1>
    
</div>


## Before Running the notebook

Please complete [setting up python dev environment](./setup-python-dev-env.md)

## Overview

This notebook will process PDF documents as part of RAG pipeline

![](media/rag-overview-2.png)

This notebook will perform steps 1, 2 and 3 in RAG pipeline.

Here are the processing steps:

Here are the processing steps:

- **pdf2parquet** : Extract text (in markdown format) from PDF and store them as parquet files
- **Exact Dedup**: Documents with exact content are filtered out
- **Chunk documents**: Split the PDFs into 'meaningful sections' (paragraphs, sentences ..etc)
- **Text encoder**: Convert chunks into vectors using embedding models

## Step-1: Configuration

In [1]:
import os
from my_config import MY_CONFIG

## RAY CONFIGURATION
num_cpus_available =  os.cpu_count()
# print (num_cpus_available)
# MY_CONFIG.RAY_NUM_CPUS = num_cpus_available // 2  ## use half the available cores for processing
MY_CONFIG.RAY_NUM_CPUS =  0.5
MY_CONFIG.RAY_MEMORY_GB = 2  # GB
# MY_CONFIG.RAY_RUNTIME_WORKERS = num_cpus_available // 3
MY_CONFIG.RAY_RUNTIME_WORKERS = 2

print (f"Ray configuration: CPUs={MY_CONFIG.RAY_NUM_CPUS},   memory={MY_CONFIG.RAY_MEMORY_GB} GB,  workers={MY_CONFIG.RAY_RUNTIME_WORKERS}")

Ray configuration: CPUs=0.5,   memory=2 GB,  workers=2


## Step-2:  Data

We will use white papers  about LLMs.  

- [Granite Code Models](https://arxiv.org/abs/2405.04324)
- [Attention is all you need](https://arxiv.org/abs/1706.03762)

You can of course substite your own data below

### 2.1 - Download data

In [2]:
import os, sys
import shutil
from utils import download_file

shutil.rmtree(MY_CONFIG.INPUT_DATA_DIR, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.INPUT_DATA_DIR, exist_ok=True)
print ("✅ Cleared input directory")
 
download_file (url = 'https://arxiv.org/pdf/1706.03762', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'attention.pdf' ))
download_file (url = 'https://arxiv.org/pdf/2405.04324', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'granite.pdf' ))
download_file (url = 'https://arxiv.org/pdf/2405.04324', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'granite2.pdf' )) # duplicate


✅ Cleared input directory

input/attention.pdf (2.22 MB) downloaded successfully.

input/granite.pdf (1.27 MB) downloaded successfully.

input/granite2.pdf (1.27 MB) downloaded successfully.


### 2.2  - Set input/output path variables for the pipeline

In [3]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DATA_DIR ):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")

output_parquet_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '01_parquet_out')
output_exact_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '02_dedupe_out')
output_chunk_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '03_chunk_out')
output_embeddings_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '04_embeddings_out')


## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


## Step-3: pdf2parquet -  Convert data from PDF to Parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### 3.1 -  Execute 

In [4]:
%%time 

from dpk_pdf2parquet.ray.transform import Pdf2Parquet
from data_processing.utils import GB
from dpk_pdf2parquet.transform import pdf2parquet_contents_types

STAGE = 1 
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{MY_CONFIG.INPUT_DATA_DIR}' --> output='{output_parquet_dir}'\n", flush=True)

result = Pdf2Parquet(input_folder= MY_CONFIG.INPUT_DATA_DIR,
                    output_folder= output_parquet_dir, 
                    data_files_to_use=['.pdf'],
                    pdf2parquet_contents_type=pdf2parquet_contents_types.MARKDOWN,
                    # pdf2parquet_contents_type=pdf2parquet_contents_types.JSON,
                    
                    ## runtime options
                    run_locally= True,
                    num_cpus= MY_CONFIG.RAY_NUM_CPUS,
                    memory= MY_CONFIG.RAY_MEMORY_GB * GB,
                    runtime_num_workers = MY_CONFIG.RAY_RUNTIME_WORKERS,

                    ## debug
                    # num_cpus=  1, 
                    # memory= MY_CONFIG.RAY_MEMORY_GB * GB, 
                    # runtime_num_workers = 1, ## Note: has to be one for this particular job, to prevent race condition when downloading models!
               ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-1: Processing input='input' --> output='output/01_parquet_out'



22:47:52 INFO - pdf2parquet parameters are : {'batch_size': -1, 'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.MARKDOWN: 'text/markdown'>, 'do_table_structure': True, 'do_ocr': True, 'ocr_engine': <pdf2parquet_ocr_engine.EASYOCR: 'easyocr'>, 'bitmap_area_threshold': 0.05, 'pdf_backend': <pdf2parquet_pdf_backend.DLPARSE_V2: 'dlparse_v2'>, 'double_precision': 8}
22:47:52 INFO - pipeline id pipeline_id
22:47:52 INFO - code location None
22:47:52 INFO - number of workers 2 worker options {'num_cpus': 0.5, 'memory': 2147483648, 'max_restarts': -1}
22:47:52 INFO - actor creation delay 0
22:47:52 INFO - job details {'job category': 'preprocessing', 'job name': 'pdf2parquet', 'job type': 'ray', 'job id': 'job_id'}
22:47:52 INFO - data factory data_ is using local data access: input_folder - input output_folder - output/01_parquet_out
22:47:52 INFO - data factory data_ max_files -1, n_sample -1
22:47:52 INFO - data factory data_ Not using data sets, checkpointing False, ma

(RayTransformFileProcessor pid=1162792) ERR#: COULD NOT CONVERT TO RS THIS TABLE TO COMPUTE SPANS


(orchestrate pid=1161876) 22:48:57 INFO - Completed 1 files in 0.807 min
(orchestrate pid=1161876) 22:48:57 INFO - Completed 1 files (33.333%)  in 0.807 min. Waiting for completion
(RayTransformFileProcessor pid=1162791) 22:48:01 INFO - Initializing models
Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 69391.06it/s]
(RayTransformFileProcessor pid=1162791) Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
(orchestrate pid=1161876) 22:51:42 INFO - Completed processing 3 files in 3.571 min
(orchestrate pid=1161876) 22:51:42 INFO - done flushing in 0.001 sec
22:51:52 INFO - Completed execution in 4.009 min, execution result 0


✅ Stage:1 completed successfully
CPU times: user 4.25 s, sys: 783 ms, total: 5.03 s
Wall time: 4min 5s


### 3.2 - Inspect Generated output

Here we should see one entry per input file processed

In [5]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_parquet_dir)
# print ("Output dimensions (rows x columns)= ", output_df.shape)
output_df.head(5)
## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,attention.pdf,"Provided proper attribution is provided, Googl...",15,6,147,0677706d-c587-4ddc-a52d-7ed12b082cbe,2949302674760005271,pdf,f1f600333e46c5d7e23f5a110a903ee38aab0bf7047eca...,46040,2025-01-19T22:48:56.992519,48.361864,attention.pdf
1,granite2.pdf,## Granite Code Models: A Family of Open Found...,28,19,295,ab9c2476-0e95-4b0e-84c9-1efab49761de,3127757990743433032,pdf,0534b6a29ab9cedf21c3f6cf756cf0252d19a8e9135a41...,127678,2025-01-19T22:51:42.875845,165.833343,granite2.pdf
2,granite.pdf,## Granite Code Models: A Family of Open Found...,28,19,295,20953ad1-8227-4454-8b20-c412f5520185,3127757990743433032,pdf,0534b6a29ab9cedf21c3f6cf756cf0252d19a8e9135a41...,127678,2025-01-19T22:50:57.694515,169.037999,granite.pdf


## Step-4: Eliminate Duplicate Documents

We have 2 duplicate documnets here : `granite.pdf` and `granite2.pdf`.

Note how the `hash` for these documents are same.

We are going to perform **de-dupe**

On the content of each document, a SHA256 hash is computed, followed by de-duplication of record having identical hashes.

[Dedupe transform documentation](https://github.com/IBM/data-prep-kit/blob/dev/transforms/universal/ededup/README.md)

### 4.1 - Execute 

In [6]:
%%time 

from dpk_ededup.ray.transform import Ededup

STAGE = 2
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_parquet_dir}' --> output='{output_exact_dedupe_dir}'\n", flush=True)

result = Ededup(input_folder=output_parquet_dir,
                output_folder=output_exact_dedupe_dir,
                ededup_hash_cpu= 0.5,
                ededup_num_hashes= 2,
                ededup_doc_column="contents",
                ededup_doc_id_column="document_id",
                
                ## runtime options
                run_locally= True,
                num_cpus= MY_CONFIG.RAY_NUM_CPUS,
                memory= MY_CONFIG.RAY_MEMORY_GB * GB,
                runtime_num_workers = MY_CONFIG.RAY_RUNTIME_WORKERS,
    ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-2: Processing input='output/01_parquet_out' --> output='output/02_dedupe_out'



22:51:54 INFO - exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'document_id', 'use_snapshot': False, 'snapshot_directory': None, 'hash_cpu': 0.5, 'num_hashes': 2}
22:51:54 INFO - pipeline id pipeline_id
22:51:54 INFO - code location None
22:51:54 INFO - number of workers 2 worker options {'num_cpus': 0.5, 'memory': 2147483648, 'max_restarts': -1}
22:51:54 INFO - actor creation delay 0
22:51:54 INFO - job details {'job category': 'preprocessing', 'job name': 'ededup', 'job type': 'ray', 'job id': 'job_id'}
22:51:54 INFO - data factory data_ is using local data access: input_folder - output/01_parquet_out output_folder - output/02_dedupe_out
22:51:54 INFO - data factory data_ max_files -1, n_sample -1
22:51:54 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
22:51:54 INFO - Running locally
2025-01-19 22:51:55,430	INFO worker.py:1777 -- Started a local Ray in

✅ Stage:2 completed successfully
CPU times: user 93.2 ms, sys: 141 ms, total: 234 ms
Wall time: 14.9 s


### 4.2 - Inspect Generated output

We would see 2 documents: `attention.pdf`  and `granite.pdf`.  The duplicate `granite.pdf` has been filtered out!

In [7]:
from utils import read_parquet_files_as_df

input_df = read_parquet_files_as_df(output_parquet_dir)
output_df = read_parquet_files_as_df(output_exact_dedupe_dir)

# print ("Input data dimensions (rows x columns)= ", input_df.shape)
# print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input files before exact dedupe : {input_df.shape[0]:,}")
print (f"Output files after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate files removed :  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input files before exact dedupe : 3
Output files after exact dedupe : 2
Duplicate files removed :   1


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,removed
0,attention.pdf,"Provided proper attribution is provided, Googl...",15,6,147,0677706d-c587-4ddc-a52d-7ed12b082cbe,2949302674760005271,pdf,f1f600333e46c5d7e23f5a110a903ee38aab0bf7047eca...,46040,2025-01-19T22:48:56.992519,48.361864,attention.pdf,[]
1,granite.pdf,## Granite Code Models: A Family of Open Found...,28,19,295,20953ad1-8227-4454-8b20-c412f5520185,3127757990743433032,pdf,0534b6a29ab9cedf21c3f6cf756cf0252d19a8e9135a41...,127678,2025-01-19T22:50:57.694515,169.037999,granite.pdf,[]


##  Step-5: Doc chunks

Split the documents in chunks.

[Chunking transform documentation](https://github.com/IBM/data-prep-kit/blob/dev/transforms/language/doc_chunk/README.md)

**Experiment with chunking size to find the setting that works best for your documents**

### 5.1 -  Execute 

In [8]:
%%time

from dpk_doc_chunk.ray.transform import DocChunk
from data_processing.utils import GB

STAGE = 3
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_exact_dedupe_dir}' --> output='{output_chunk_dir}'\n", flush=True)

result = DocChunk(input_folder=output_exact_dedupe_dir,
                        output_folder=output_chunk_dir,
                        doc_chunk_chunking_type= "li_markdown",

                        ## runtime options
                        run_locally= True,
                        num_cpus= MY_CONFIG.RAY_NUM_CPUS,
                        memory= MY_CONFIG.RAY_MEMORY_GB * GB,
                        runtime_num_workers = MY_CONFIG.RAY_RUNTIME_WORKERS,
                        ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-3: Processing input='output/02_dedupe_out' --> output='output/03_chunk_out'



22:52:10 INFO - doc_chunk parameters are : {'chunking_type': 'li_markdown', 'content_column_name': 'contents', 'doc_id_column_name': 'document_id', 'output_chunk_column_name': 'contents', 'output_source_doc_id_column_name': 'source_document_id', 'output_jsonpath_column_name': 'doc_jsonpath', 'output_pageno_column_name': 'page_number', 'output_bbox_column_name': 'bbox', 'chunk_size_tokens': 128, 'chunk_overlap_tokens': 30, 'dl_min_chunk_len': None}
22:52:10 INFO - pipeline id pipeline_id
22:52:10 INFO - code location None
22:52:10 INFO - number of workers 2 worker options {'num_cpus': 0.5, 'memory': 2147483648, 'max_restarts': -1}
22:52:10 INFO - actor creation delay 0
22:52:10 INFO - job details {'job category': 'preprocessing', 'job name': 'doc_chunk', 'job type': 'ray', 'job id': 'job_id'}
22:52:10 INFO - data factory data_ is using local data access: input_folder - output/02_dedupe_out output_folder - output/03_chunk_out
22:52:10 INFO - data factory data_ max_files -1, n_sample -1
2

✅ Stage:3 completed successfully
CPU times: user 1.03 s, sys: 311 ms, total: 1.34 s
Wall time: 18.3 s


### 5.2 - Inspect Generated output

We would see documents are split into many chunks

In [9]:
from utils import read_parquet_files_as_df

input_df = read_parquet_files_as_df(output_exact_dedupe_dir)  ## for debug purposes
output_df = read_parquet_files_as_df(output_chunk_dir)

print (f"Files processed : {input_df.shape[0]:,}")
print (f"Chunks created : {output_df.shape[0]:,}")

# print ("Input data dimensions (rows x columns)= ", input_df.shape)
# print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Files processed : 2
Chunks created : 60


,filename,num_pages,num_tables,num_doc_elements,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,removed,source_document_id,contents,document_id
1,attention.pdf,15,6,147,2949302674760005271,pdf,f1f600333e46c5d7e23f5a110a903ee38aab0bf7047eca...,46040,2025-01-19T22:48:56.992519,48.361864,attention.pdf,[],0677706d-c587-4ddc-a52d-7ed12b082cbe,## Attention Is All You Need\n\nAshish Vaswani...,45e678f43369d5fa127105b7cca6a6e4dd4deed6422185...
58,granite.pdf,28,19,295,3127757990743433032,pdf,0534b6a29ab9cedf21c3f6cf756cf0252d19a8e9135a41...,127678,2025-01-19T22:50:57.694515,169.037999,granite.pdf,[],20953ad1-8227-4454-8b20-c412f5520185,"## References\n\nWasi Uddin Ahmad, Md Golam Ra...",b787f46ab644038e472b9815a122eead379ed7f37a3d4f...
52,granite.pdf,28,19,295,3127757990743433032,pdf,0534b6a29ab9cedf21c3f6cf756cf0252d19a8e9135a41...,127678,2025-01-19T22:50:57.694515,169.037999,granite.pdf,[],20953ad1-8227-4454-8b20-c412f5520185,"## 6.4 Code Reasoning, Understanding and Execu...",1c7f5e76a2aaad73f5f03549b065016b0703239538839d...


## Step-6:   Calculate Embeddings for Chunks

we will calculate embeddings for each chunk using an open source embedding model

[Embeddings / Text Encoder documentation](https://github.com/IBM/data-prep-kit/blob/dev/transforms/language/text_encoder/README.md)

### 6.1 - Execute

In [10]:
%%time 

from dpk_text_encoder.ray.transform import TextEncoder
from data_processing.utils import GB

STAGE  = 4
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_chunk_dir}' --> output='{output_embeddings_dir}'\n", flush=True)

result = TextEncoder(input_folder= output_chunk_dir, 
               output_folder= output_embeddings_dir, 
               text_encoder_model_name = MY_CONFIG.EMBEDDING_MODEL,
               
               ## runtime options
               run_locally= True,
               num_cpus= MY_CONFIG.RAY_NUM_CPUS,
               memory= MY_CONFIG.RAY_MEMORY_GB * GB,
               runtime_num_workers = MY_CONFIG.RAY_RUNTIME_WORKERS,
               ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-4: Processing input='output/03_chunk_out' --> output='output/04_embeddings_out'



22:52:28 INFO - text_encoder parameters are : {'content_column_name': 'contents', 'output_embeddings_column_name': 'embeddings', 'model_name': 'sentence-transformers/all-MiniLM-L6-v2'}
22:52:28 INFO - pipeline id pipeline_id
22:52:28 INFO - code location None
22:52:28 INFO - number of workers 2 worker options {'num_cpus': 0.5, 'memory': 2147483648, 'max_restarts': -1}
22:52:28 INFO - actor creation delay 0
22:52:28 INFO - job details {'job category': 'preprocessing', 'job name': 'text_encoder', 'job type': 'ray', 'job id': 'job_id'}
22:52:28 INFO - data factory data_ is using local data access: input_folder - output/03_chunk_out output_folder - output/04_embeddings_out
22:52:28 INFO - data factory data_ max_files -1, n_sample -1
22:52:28 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
22:52:28 INFO - Running locally
2025-01-19 22:52:29,668	INFO worker.py:1777 -- Started a lo

✅ Stage:4 completed successfully
CPU times: user 674 ms, sys: 298 ms, total: 972 ms
Wall time: 26.2 s


### 6.2 - Inspect Generated output

In [11]:
from utils import read_parquet_files_as_df

input_df = read_parquet_files_as_df(output_chunk_dir)
output_df = read_parquet_files_as_df(output_embeddings_dir)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (60, 15)
Output data dimensions (rows x columns)=  (60, 16)


,filename,num_pages,num_tables,num_doc_elements,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,removed,source_document_id,contents,document_id,embeddings
44,granite.pdf,28,19,295,3127757990743433032,pdf,0534b6a29ab9cedf21c3f6cf756cf0252d19a8e9135a41...,127678,2025-01-19T22:50:57.694515,169.037999,granite.pdf,[],20953ad1-8227-4454-8b20-c412f5520185,## 6.1.1 HumanEvalSynthesize: Multilingual Cod...,b10bcf46720fb7fff15818c4bc03ec37ae84181e6cbbc1...,"[-0.03851807, 0.00934296, 0.02425409, -0.00439..."
15,attention.pdf,15,6,147,2949302674760005271,pdf,f1f600333e46c5d7e23f5a110a903ee38aab0bf7047eca...,46040,2025-01-19T22:48:56.992519,48.361864,attention.pdf,[],0677706d-c587-4ddc-a52d-7ed12b082cbe,## 5 Training\n\nThis section describes the tr...,7e7dce074e6995e9c9551e1349cad58153b319c45e20a1...,"[-0.02469791, -0.077463716, 0.07508141, 0.0363..."
40,granite.pdf,28,19,295,3127757990743433032,pdf,0534b6a29ab9cedf21c3f6cf756cf0252d19a8e9135a41...,127678,2025-01-19T22:50:57.694515,169.037999,granite.pdf,[],20953ad1-8227-4454-8b20-c412f5520185,## 4.4 Infrastructure\n\nWe train the Granite ...,81f51d21fd61607d8aa9bb50925c2bc936fa1da7b27f4b...,"[-0.033672214, -0.01862875, 0.0034308454, 0.06..."


## Step-7: Copy output to final output dir

In [12]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.copytree(src=output_embeddings_dir, dst=MY_CONFIG.OUTPUT_FOLDER_FINAL)

print (f"✅ Copied output from '{output_embeddings_dir}' --> '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")

✅ Copied output from 'output/04_embeddings_out' --> 'output/output_final'
